In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.tree import DecisionTreeRegressor

from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [52]:
event = pd.read_csv('events.csv')
event.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
#time consuming steps
event.timestamp = event.timestamp.apply(lambda x: pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))

In [7]:
# imbalanced view and transaction
event.event.value_counts()

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [53]:
# number of customers purchased items
customer_purchased = event[event.transactionid.notnull()].visitorid.unique()
customer_purchased.size

11719

In [10]:
event.to_csv('events_dates.csv', index=False)

In [54]:
events = pd.read_csv('events_dates.csv')
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 10:32:12,257597,view,355908,NaN
1,2015-06-02 11:20:14,992329,view,248676,NaN
2,2015-06-02 10:43:19,111016,view,318965,NaN
3,2015-06-02 10:42:35,483717,view,253185,NaN
4,2015-06-02 10:32:17,951259,view,367447,NaN


## 1.2 Item

In [12]:
item_1 = pd.read_csv('item_properties_part1.csv')

In [5]:
item_2 = pd.read_csv('item_properties_part2.csv')

In [6]:
# concat two tables into one
items = pd.concat([item_1, item_2], axis=0)
items.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [11]:
# converting Unix timestamp to readable format, time consuming steps
items.timestamp = items.timestamp.apply(lambda x: pd.to_datetime(datetime.fromtimestamp(x//1000).strftime('%Y-%m-%d %H:%M:%S')))

In [13]:
# save to csv
items.to_csv('items.csv', index=False)

In [13]:
# read items csv
items = pd.read_csv('items.csv')
items.head()

,timestamp,itemid,property,value
0,2015-06-28 08:30:00,460429,categoryid,1338
1,2015-09-06 08:30:00,206783,888,1116713 960601 n277.200
2,2015-08-09 08:30:00,395014,400,n552.000 639502 n720.000 424566
3,2015-05-10 08:30:00,59481,790,n15360.000
4,2015-05-17 08:30:00,156781,917,828513


## 1.3 Category

In [14]:
category = pd.read_csv('category_tree.csv')
category.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [20]:
# 25 null value in parentid
category.isnull().value_counts()

categoryid  parentid
False       False       1644
            True          25
Name: count, dtype: int64

In [23]:
# items with categoryid and available as propery
items = items.loc[items.property.isin(['categoryid', 'available']), :]
print("items with categoryid and available as propery:", items.size)
items.head()

items with categoryid and available as propery: 9167412


,timestamp,itemid,property,value
0,2015-06-28 08:30:00,460429,categoryid,1338
5,2015-07-05 08:30:00,285026,available,0
15,2015-07-19 08:30:00,186518,available,0
79,2015-06-07 08:30:00,423682,available,0
82,2015-06-14 08:30:00,316253,available,1


In [24]:
# grouping itemid by its event type and creating list of each of them
grouped = events.groupby('event')['itemid'].apply(list)
grouped

event
addtocart      [5206, 10572, 255275, 356475, 312728, 63312, 1...
transaction    [356475, 15335, 81345, 150318, 310791, 54058, ...
view           [355908, 248676, 318965, 253185, 367447, 22556...
Name: itemid, dtype: object

In [55]:
# all unique visitors
all_customers = events['visitorid'].unique()
print("Unique visitors:", all_customers.size)

# all visitors
print('Total visitors:', events['visitorid'].size)

Unique visitors: 1407580
Total visitors: 2756101


In [56]:
# get all the customers who bought something
customer_purchased = events[events.transactionid.notnull()].visitorid.unique()
customer_purchased.size

11719

In [57]:
# get visitors who didn't buy anything
customer_browsed = [x for x in all_customers if x not in customer_purchased]
len(customer_browsed)

1395861

In [28]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 10:32:12,257597,view,355908,NaN
1,2015-06-02 11:20:14,992329,view,248676,NaN
2,2015-06-02 10:43:19,111016,view,318965,NaN
3,2015-06-02 10:42:35,483717,view,253185,NaN
4,2015-06-02 10:32:17,951259,view,367447,NaN


In [30]:
items.head()

,timestamp,itemid,property,value
0,2015-06-28 08:30:00,460429,categoryid,1338
5,2015-07-05 08:30:00,285026,available,0
15,2015-07-19 08:30:00,186518,available,0
79,2015-06-07 08:30:00,423682,available,0
82,2015-06-14 08:30:00,316253,available,1


In [31]:
# create items_ that filtered the unique itemid with categoryid value
items_ = items[items.property == 'categoryid'].drop(['timestamp', 'property'], axis=1).drop_duplicates()
items_.head()

,itemid,value
0,460429,1338
140,281245,1277
151,35575,1059
189,8313,1147
197,55102,47


In [32]:
# convert object to int64 for value in items_, perpare for merge table
items_.value = items_.value.astype('int64')

In [33]:
category.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [34]:
# check the datatype of category
category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [35]:
# create the merged data with categoryid and parentid for items in events dataframe
data = events.merge(items_, on='itemid').merge(category, left_on='value', right_on='categoryid').drop('value', axis=1)
data.head()

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid
0,2015-06-02 10:32:12,257597,view,355908,NaN,1173,805.0
1,2015-06-02 11:20:14,992329,view,248676,NaN,1231,901.0
2,2015-06-02 10:42:35,483717,view,253185,NaN,914,226.0
3,2015-06-02 10:32:17,951259,view,367447,NaN,1613,250.0
4,2015-06-02 10:32:17,951259,view,367447,NaN,491,679.0


In [36]:
data.to_csv('data_new.csv', index = False)

### 2.3.1 EDA of summarized data

In [2]:
data = pd.read_csv('data_new.csv')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2705278 entries, 0 to 2709445
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      datetime64[ns]
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  float64       
 5   categoryid     int64         
 6   parentid       float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 165.1+ MB


In [5]:
# convert the datatype
data.timestamp = data.timestamp.astype('datetime64[ns]')

In [7]:
data = data.drop_duplicates()

In [12]:
data.transactionid.isnull().value_counts()

transactionid
True     2681915
False      23363
Name: count, dtype: int64

In [10]:
data.event.value_counts()

event
view           2610352
addtocart        71563
transaction      23363
Name: count, dtype: int64

In [13]:
data['transactionid'].value_counts()

transactionid
7063.0     34
765.0      30
8351.0     27
2753.0     27
6993.0     21
           ..
14021.0     1
17414.0     1
3999.0      1
3786.0      1
17579.0     1
Name: count, Length: 17313, dtype: int64

In [15]:
data.describe()

,timestamp,visitorid,itemid,transactionid,categoryid,parentid
count,2705278,2.705278e+06,2.705278e+06,23363.000000,2.705278e+06,2.705260e+06
mean,2015-07-09 04:54:55.313824768,7.016354e+05,2.348950e+05,8832.780294,8.773571e+02,8.669219e+02
min,2015-05-03 08:30:04,0.000000e+00,3.000000e+00,0.000000,0.000000e+00,8.000000e+00
25%,2015-06-05 04:35:32.249999872,3.500890e+05,1.182150e+05,4419.500000,4.730000e+02,4.400000e+02
50%,2015-07-09 06:10:35.500000,7.015695e+05,2.359020e+05,8827.000000,9.580000e+02,8.710000e+02
75%,2015-08-10 04:05:39.750000128,1.052846e+06,3.506290e+05,13222.000000,1.279000e+03,1.329000e+03
max,2015-09-18 08:29:47,1.407579e+06,4.668640e+05,17671.000000,1.697000e+03,1.698000e+03
std,NaN,4.056618e+05,1.340439e+05,5100.994651,4.867197e+02,5.068626e+02


### 2.3.2 Timestamp

In [17]:
data.sort_values(by=['visitorid','itemid', 'timestamp'])

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid
1338798,2015-09-12 02:25:17,0,view,67045,NaN,333,1497.0
1333375,2015-09-12 02:19:49,0,view,285930,NaN,1188,1497.0
1338670,2015-09-12 02:22:39,0,view,357564,NaN,256,1257.0
817585,2015-08-13 23:16:06,1,view,72028,NaN,1192,955.0
717304,2015-08-07 23:31:08,2,view,216305,NaN,299,73.0
...,...,...,...,...,...,...,...
204263,2015-06-11 03:16:08,1407575,view,121220,NaN,1221,1426.0
47148,2015-06-03 20:31:29,1407576,view,356208,NaN,175,1492.0
1728728,2015-05-18 03:18:04,1407577,view,427784,NaN,1237,623.0
1710517,2015-05-17 06:51:23,1407578,view,188736,NaN,1584,1562.0


In [21]:
item_tra=data[['visitorid','itemid','timestamp']][data['event']=='transaction']
item_atc=data[['visitorid','itemid','timestamp']][data['event']=='addtocart']
item_viw=data[['visitorid','itemid','timestamp']][data['event']=='view']

In [22]:
# create a dataframe of visitor, itemid found in all three events
time_df=item_tra.merge(item_atc, how='inner', on=['visitorid','itemid'], suffixes=[' (transaction)', ' (add_to_cart)'])
time_df=time_df.merge(item_viw, how='inner', on=['visitorid','itemid'])
time_df=time_df.rename(columns={'timestamp':'timestamp (view)'})
time_df.head()

,visitorid,itemid,timestamp (transaction),timestamp (add_to_cart),timestamp (view)
0,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-02 10:41:36
1,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-03 07:42:25
2,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-07 02:40:10
3,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-08 08:14:57
4,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-10 06:04:37


In [23]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116294 entries, 0 to 116293
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   visitorid                116294 non-null  int64         
 1   itemid                   116294 non-null  int64         
 2   timestamp (transaction)  116294 non-null  datetime64[ns]
 3   timestamp (add_to_cart)  116294 non-null  datetime64[ns]
 4   timestamp (view)         116294 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(2)
memory usage: 4.4 MB


In [24]:
# calculate the time differences
time_df['cart_to_transaction'] = (time_df['timestamp (transaction)'] - time_df['timestamp (add_to_cart)']).apply(lambda x: x.total_seconds()/3600)
time_df['first_view'] = time_df.groupby('itemid')['timestamp (view)'].transform('min')
time_df['firstview_to_cart'] = (time_df['timestamp (add_to_cart)'] - time_df['first_view']).apply(lambda x: x.total_seconds()/3600)

time_df.head()

,visitorid,itemid,timestamp (transaction),timestamp (add_to_cart),timestamp (view),cart_to_transaction,first_view,firstview_to_cart
0,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-02 10:41:36,0.093056,2015-06-02 10:41:36,0.0125
1,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-03 07:42:25,0.093056,2015-06-02 10:41:36,0.0125
2,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-07 02:40:10,0.093056,2015-06-02 10:41:36,0.0125
3,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-08 08:14:57,0.093056,2015-06-02 10:41:36,0.0125
4,599528,356475,2015-06-02 10:47:56,2015-06-02 10:42:21,2015-06-10 06:04:37,0.093056,2015-06-02 10:41:36,0.0125


### 2.3.3 Category

In [32]:
data['purchased'] = data['transactionid'].apply(lambda x: 1 if not pd.isnull(x) else 0)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2705278 entries, 0 to 2709445
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      datetime64[ns]
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  float64       
 5   categoryid     int64         
 6   parentid       float64       
 7   purchased      int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 185.8+ MB


In [34]:
data.describe()

,timestamp,visitorid,itemid,transactionid,categoryid,parentid,purchased
count,2705278,2.705278e+06,2.705278e+06,23363.000000,2.705278e+06,2.705260e+06,2.705278e+06
mean,2015-07-09 04:54:55.313824768,7.016354e+05,2.348950e+05,8832.780294,8.773571e+02,8.669219e+02,8.636081e-03
min,2015-05-03 08:30:04,0.000000e+00,3.000000e+00,0.000000,0.000000e+00,8.000000e+00,0.000000e+00
25%,2015-06-05 04:35:32.249999872,3.500890e+05,1.182150e+05,4419.500000,4.730000e+02,4.400000e+02,0.000000e+00
50%,2015-07-09 06:10:35.500000,7.015695e+05,2.359020e+05,8827.000000,9.580000e+02,8.710000e+02,0.000000e+00
75%,2015-08-10 04:05:39.750000128,1.052846e+06,3.506290e+05,13222.000000,1.279000e+03,1.329000e+03,0.000000e+00
max,2015-09-18 08:29:47,1.407579e+06,4.668640e+05,17671.000000,1.697000e+03,1.698000e+03,1.000000e+00
std,NaN,4.056618e+05,1.340439e+05,5100.994651,4.867197e+02,5.068626e+02,9.252839e-02


In [36]:
ds = data.copy()
ds.head()

,timestamp,visitorid,event,itemid,transactionid,categoryid,parentid,purchased
0,2015-06-02 10:32:12,257597,view,355908,NaN,1173,805.0,0
1,2015-06-02 11:20:14,992329,view,248676,NaN,1231,901.0,0
2,2015-06-02 10:42:35,483717,view,253185,NaN,914,226.0,0
3,2015-06-02 10:32:17,951259,view,367447,NaN,1613,250.0,0
4,2015-06-02 10:32:17,951259,view,367447,NaN,491,679.0,0


In [38]:
ds['purchased'].value_counts()

purchased
0    2681915
1      23363
Name: count, dtype: int64

In [39]:
ds = ds.drop(['timestamp', 'transactionid'], axis = 1)

In [40]:
# views_num of each item by each visitor
ds['item_visitor_views_num'] = ds[ds['event']=='view'].groupby(['visitorid', 'itemid'])['event'].transform('count')

In [43]:
# whether the item added to cart
ds['add_to_cart'] = ds['event'].apply(lambda x: 1 if x == 'addtocart' else 0)

In [44]:
ds.head()

,visitorid,event,itemid,categoryid,parentid,purchased,item_visitor_views_num,add_to_cart
0,257597,view,355908,1173,805.0,0,1.0,0
1,992329,view,248676,1231,901.0,0,1.0,0
2,483717,view,253185,914,226.0,0,2.0,0
3,951259,view,367447,1613,250.0,0,2.0,0
4,951259,view,367447,491,679.0,0,2.0,0


In [45]:
# remove data that visitor purchased same item multipule times
ds = ds.drop_duplicates()

In [46]:
# convert categorical datatype to category
ds['visitorid'] = ds['visitorid'].astype('category')
ds['itemid'] = ds['itemid'].astype('category')
ds['categoryid'] = ds['categoryid'].astype('category')
ds['parentid'] = ds['parentid'].astype('category')
ds['event'] = ds['event'].astype('category')
ds['add_to_cart'] = ds['add_to_cart'].astype('category')
ds['purchased'] = ds['purchased'].astype('category')

In [47]:
# drop visitorid and duplicates
ds = ds.drop('visitorid', axis=1).drop_duplicates()

In [48]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 361663 entries, 0 to 2709403
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   event                   361663 non-null  category
 1   itemid                  361663 non-null  category
 2   categoryid              361663 non-null  category
 3   parentid                361660 non-null  category
 4   purchased               361663 non-null  category
 5   item_visitor_views_num  324154 non-null  float64 
 6   add_to_cart             361663 non-null  category
dtypes: category(6), float64(1)
memory usage: 14.8 MB


In [49]:
ds.itemid.nunique()

185246

In [50]:
ds[ds['item_visitor_views_num'].isnull()]

,event,itemid,categoryid,parentid,purchased,item_visitor_views_num,add_to_cart
18,addtocart,5206,927,605.0,0,NaN,1
19,addtocart,5206,381,1482.0,0,NaN,1
21,addtocart,10572,1037,402.0,0,NaN,1
64,addtocart,255275,1002,879.0,0,NaN,1
111,addtocart,356475,386,1095.0,0,NaN,1
...,...,...,...,...,...,...,...
2708528,addtocart,409951,1404,593.0,0,NaN,1
2708643,transaction,446271,799,920.0,1,NaN,0
2708793,addtocart,59290,1421,105.0,0,NaN,1
2708896,addtocart,430566,746,1200.0,0,NaN,1


In [61]:
def create_dataframe(visitor_list):
    df_array = []
    for index in visitor_list:
        v_df = data[data.visitorid == index]
        m_df = time_df[time_df.visitorid == index]
        
        temp = []
        temp.append(index)
        
        temp.append(v_df[v_df.event == 'view'].itemid.unique().size)
        
        temp.append(v_df[v_df.event == 'view'].event.count())

        temp.append(m_df['firstview_to_cart'].mean())
        
        temp.append(m_df['cart_to_transaction'].mean())
        
        number_of_items_bought = v_df[v_df.event == 'transaction'].event.count()
        temp.append(number_of_items_bought)
        
        if(number_of_items_bought == 0):
            temp.append(0)
        else:
            temp.append(1)

        df_array.append(temp)
        
    return pd.DataFrame(df_array, columns=['visitorid', 'num_items_viewed', 'view_count', 'firstview_to_cart', 'cart_to_transaction', 'bought_count', 'purchased'])

In [62]:
buying_visitors_df = create_dataframe(customer_purchased)
buying_visitors_df.head()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased
0,599528,2,15,0.012500,0.093056,1,1
1,121688,13,16,-0.145082,0.927173,12,1
2,552148,1,1,0.036667,0.009722,1,1
3,102019,2,6,0.024167,0.259722,2,1
4,189384,7,25,479.378596,0.051974,2,1


In [63]:
buying_visitors_df.shape

(11719, 7)

In [64]:
import random
random.shuffle(customer_browsed)

In [142]:

subset_size = int(len(customer_browsed) * 1)
subset = random.sample(customer_browsed, subset_size)

viewing_visitors_df = create_dataframe(subset)



In [143]:
data_ml = pd.concat([buying_visitors_df, viewing_visitors_df], ignore_index=True)

In [144]:
data_ml.describe()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased
count,1.407580e+06,1.407580e+06,1.407580e+06,9791.000000,9791.000000,1.407580e+06,1.407580e+06
mean,7.037895e+05,1.358005e+00,1.854496e+00,428.554127,3.279050,1.659799e-02,8.219071e-03
std,4.063335e+05,6.519558e+00,1.083521e+01,690.992677,39.748556,8.471507e-01,9.028579e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,-595.400764,-1478.021732,0.000000e+00,0.000000e+00
25%,3.518948e+05,1.000000e+00,1.000000e+00,0.055000,0.046319,0.000000e+00,0.000000e+00
50%,7.037895e+05,1.000000e+00,1.000000e+00,21.626667,0.089444,0.000000e+00,0.000000e+00
75%,1.055684e+06,1.000000e+00,2.000000e+00,627.864167,0.219056,0.000000e+00,0.000000e+00
max,1.407579e+06,3.280000e+03,5.722000e+03,3276.660278,1198.972500,5.750000e+02,1.000000e+00


In [145]:
# shuffle main_df first
data_ml = data_ml.sample(frac=1)

In [146]:
data_ml.to_csv('newcsv.csv', index=False)

In [148]:
data_ml = pd.read_csv('newcsv.csv')
data_ml.head()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased
0,512286,1,1,NaN,NaN,0,0
1,376052,0,0,NaN,NaN,0,0
2,1063766,8,13,NaN,NaN,0,0
3,58180,4,6,NaN,NaN,0,0
4,428164,0,0,NaN,NaN,0,0


In [149]:
data_ml['view_rate'] = data_ml['num_items_viewed']/data_ml['view_count']

In [151]:

data_ml['purchase_rate'] = data_ml['bought_count']/data_ml['view_count']

In [153]:
data_ml.purchased.value_counts()

purchased
0    1396011
1      11569
Name: count, dtype: int64

In [154]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407580 entries, 0 to 1407579
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   visitorid            1407580 non-null  int64  
 1   num_items_viewed     1407580 non-null  int64  
 2   view_count           1407580 non-null  int64  
 3   firstview_to_cart    9791 non-null     float64
 4   cart_to_transaction  9791 non-null     float64
 5   bought_count         1407580 non-null  int64  
 6   purchased            1407580 non-null  int64  
 7   view_rate            1232600 non-null  float64
 8   purchase_rate        1233029 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 96.7 MB


### 3.1.1 purchase_rate, to be the label, need to drop rows in infinite value

In [156]:
# Drop rows with infinite values in the "purchase_rate" column
data_ml = data_ml[~np.isinf(data_ml['purchase_rate'])]

In [79]:
# check whether these rows be dropped
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25248 entries, 0 to 25676
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   visitorid            25248 non-null  int64  
 1   num_items_viewed     25248 non-null  int64  
 2   view_count           25248 non-null  int64  
 3   firstview_to_cart    9791 non-null   float64
 4   cart_to_transaction  9791 non-null   float64
 5   bought_count         25248 non-null  int64  
 6   purchased            25248 non-null  int64  
 7   view_rate            23452 non-null  float64
 8   purchase_rate        23452 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 1.9 MB


In [81]:
data_ml['num_items_viewed'].value_counts().head(10)

num_items_viewed
1    14129
2     3558
0     1796
3     1642
4      897
5      656
6      441
7      306
8      240
9      218
Name: count, dtype: int64

In [160]:
data_ml['view_count'].value_counts()

view_count
1      794994
2      235839
0      174551
3       75359
4       43961
        ...  
344         1
189         1
141         1
256         1
900         1
Name: count, Length: 333, dtype: int64


### 3.1.5 firstview_to_cart

In [162]:
# negetive value is due to item add_to_cart before the data collected.
data_ml[data_ml['firstview_to_cart'] < 0]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
8770,1317771,5,11,-0.005556,167.453056,1,1,0.454545,0.090909
17865,685143,1,1,-0.034722,0.015833,1,1,1.000000,1.000000
18368,1188257,10,17,-1.478657,0.235972,2,1,0.588235,0.117647
20575,569471,2,2,-0.093889,0.137222,1,1,1.000000,0.500000
20862,1234428,3,3,-0.002222,0.058056,1,1,1.000000,0.333333
...,...,...,...,...,...,...,...,...,...
1344828,396151,1,2,-1.502500,0.051389,1,1,0.500000,0.500000
1347744,663616,2,4,-0.015278,16.500000,1,1,0.500000,0.250000
1397505,983395,32,62,-595.400764,0.278750,2,1,0.516129,0.032258
1398399,586187,2,2,-0.000556,0.443333,1,1,1.000000,0.500000


In [163]:
# the max is ~4 months
data_ml.sort_values('firstview_to_cart', ascending=False).head(100)

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
384715,921915,1,1,3276.660278,0.045278,1,1,1.00,1.000000
1027198,1199182,1,1,3276.570278,0.058889,1,1,1.00,1.000000
300534,7633,1,4,3260.562222,0.142500,1,1,0.25,0.250000
524505,346832,1,1,3236.385278,0.100833,1,1,1.00,1.000000
116317,237886,2,2,3232.516944,0.068056,1,1,1.00,0.500000
...,...,...,...,...,...,...,...,...,...
604397,1294899,3,3,2786.395556,0.030833,1,1,1.00,0.333333
84603,276809,1,2,2786.351111,0.091389,2,1,0.50,1.000000
366542,618031,1,1,2781.591667,0.108056,1,1,1.00,1.000000
738566,939792,4,5,2781.492130,0.179167,2,1,0.80,0.400000


In [164]:
data_ml[data_ml['firstview_to_cart'] == 0]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
52022,533142,34,36,0.0,0.021944,1,1,0.944444,0.027778
110145,34667,3,4,0.0,0.052222,1,1,0.750000,0.250000
280917,767636,1,2,0.0,0.035833,1,1,0.500000,0.500000
379336,74832,1,2,0.0,0.056111,1,1,0.500000,0.500000
384658,1026145,1,1,0.0,0.037778,1,1,1.000000,1.000000
525511,1230189,1,2,0.0,0.026667,2,1,0.500000,1.000000
825581,1013847,1,1,0.0,0.042778,1,1,1.000000,1.000000
839036,1179416,1,1,0.0,0.085278,2,1,1.000000,2.000000
973605,20815,1,2,0.0,0.022500,1,1,0.500000,0.500000
1368447,1084182,1,1,0.0,0.026667,1,1,1.000000,1.000000


In [165]:
# null value need to be addressed
data_ml['cart_to_transaction'].isnull().value_counts()

cart_to_transaction
True     1397360
False       9791
Name: count, dtype: int64

In [166]:
# check the negetive values
data_ml[data_ml['cart_to_transaction'] < 0]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
9682,276034,16,27,13.399444,-13.202778,1,1,0.592593,0.037037
14210,668226,7,16,3.150794,-0.112540,2,1,0.437500,0.125000
16599,436380,3,5,4.022639,-3.925139,1,1,0.600000,0.200000
22679,1038651,1,4,0.725278,-0.081389,2,1,0.250000,0.500000
26223,481873,1,3,23.864444,-5.149722,1,1,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...
1359418,1362413,5,15,110.334861,-88.885278,2,1,0.333333,0.133333
1367084,1148151,3,9,71.621111,-23.794444,2,1,0.333333,0.222222
1376239,1279582,2,5,1141.964444,-0.151528,2,1,0.400000,0.400000
1385082,450190,1,8,28.983056,-28.429167,1,1,0.125000,0.125000


In [167]:
data_ml['cart_to_transaction'].min()

-1478.0217322097378

In [168]:
data_ml['cart_to_transaction'].max()

1198.9725

In [169]:
# the higher value, the longer timediff of cart to transaction made. For null values, it can be assume that cart_to_transaction is infinite.
data_ml.sort_values('cart_to_transaction', ascending=False).head(100)['purchase_rate'].mean()

0.14905489183730244

In [170]:
data_ml[(data_ml['cart_to_transaction']> -1) & (data_ml['cart_to_transaction']< 1)]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
22,1397089,1,4,1.941944,0.094444,1,1,0.250000,0.250000
60,749945,1,1,1909.540556,0.105556,1,1,1.000000,1.000000
115,855383,2,2,953.424028,0.025556,2,1,1.000000,1.000000
208,592348,2,5,0.006944,0.097500,1,1,0.400000,0.200000
471,701722,1,27,694.702130,0.166343,2,1,0.037037,0.074074
...,...,...,...,...,...,...,...,...,...
1406834,850320,2,4,0.271667,0.016111,1,1,0.500000,0.250000
1406839,1208531,7,24,0.013889,0.015000,1,1,0.291667,0.041667
1406856,1056247,3,12,23.964167,0.106944,1,1,0.250000,0.083333
1406935,284451,3,7,4.766944,0.361667,1,1,0.428571,0.142857


In [102]:
# In this case, num_items_viewed must be low
data_ml[(data_ml['view_rate'] == 1) & data_ml['purchase_rate'] != 0]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
0,1035684,1,1,0.005000,0.095000,1,1,1.0,1.0
8,1126660,1,1,1555.202222,0.045000,1,1,1.0,1.0
13,918415,1,1,-58.547500,49.907222,1,1,1.0,1.0
31,146458,1,1,NaN,NaN,1,1,1.0,1.0
32,472290,1,1,0.004167,0.982222,5,1,1.0,5.0
...,...,...,...,...,...,...,...,...,...
25615,450098,2,2,2011.006389,0.008056,1,1,1.0,0.5
25633,1228166,1,1,428.634167,0.107222,1,1,1.0,1.0
25640,1292722,2,2,0.010000,0.042222,1,1,1.0,0.5
25662,837233,1,1,0.006111,0.621111,1,1,1.0,1.0


In [173]:
data_ml[data_ml['view_rate'].isnull()]

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
1,376052,0,0,NaN,NaN,0,0,NaN,NaN
4,428164,0,0,NaN,NaN,0,0,NaN,NaN
6,1177470,0,0,NaN,NaN,0,0,NaN,NaN
31,1065275,0,0,NaN,NaN,0,0,NaN,NaN
40,1074559,0,0,NaN,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1407555,604430,0,0,NaN,NaN,0,0,NaN,NaN
1407563,354803,0,0,NaN,NaN,0,0,NaN,NaN
1407569,290630,0,0,NaN,NaN,0,0,NaN,NaN
1407573,1209143,0,0,NaN,NaN,0,0,NaN,NaN


In [174]:
data_ml.describe()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
count,1.407151e+06,1.407151e+06,1.407151e+06,9791.000000,9791.000000,1.407151e+06,1.407151e+06,1.232600e+06,1.232600e+06
mean,7.037826e+05,1.358419e+00,1.855062e+00,428.554127,3.279050,1.620295e-02,7.916705e-03,8.830757e-01,3.987579e-03
std,4.063345e+05,6.520509e+00,1.083682e+01,690.992677,39.748556,8.468844e-01,8.862300e-02,2.197732e-01,5.674046e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,-595.400764,-1478.021732,0.000000e+00,0.000000e+00,3.546099e-03,0.000000e+00
25%,3.518945e+05,1.000000e+00,1.000000e+00,0.055000,0.046319,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,7.037700e+05,1.000000e+00,1.000000e+00,21.626667,0.089444,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.055670e+06,1.000000e+00,2.000000e+00,627.864167,0.219056,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,1.407579e+06,3.280000e+03,5.722000e+03,3276.660278,1198.972500,5.750000e+02,1.000000e+00,1.000000e+00,6.000000e+00


In [175]:
# drop null value in view_rate
data_ml = data_ml.dropna(subset=['view_rate'])

In [176]:
data_ml['purchase_rate'].isnull().value_counts()

purchase_rate
False    1232600
Name: count, dtype: int64

In [177]:
data_ml.describe()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
count,1.232600e+06,1.232600e+06,1.232600e+06,9791.000000,9791.000000,1.232600e+06,1.232600e+06,1.232600e+06,1.232600e+06
mean,7.038277e+05,1.550787e+00,2.117761e+00,428.554127,3.279050,1.849748e-02,9.037806e-03,8.830757e-01,3.987579e-03
std,4.063717e+05,6.945476e+00,1.155468e+01,690.992677,39.748556,9.048408e-01,9.463684e-02,2.197732e-01,5.674046e-02
min,0.000000e+00,1.000000e+00,1.000000e+00,-595.400764,-1478.021732,0.000000e+00,0.000000e+00,3.546099e-03,0.000000e+00
25%,3.519708e+05,1.000000e+00,1.000000e+00,0.055000,0.046319,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,7.038345e+05,1.000000e+00,1.000000e+00,21.626667,0.089444,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,1.055768e+06,1.000000e+00,2.000000e+00,627.864167,0.219056,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
max,1.407579e+06,3.280000e+03,5.722000e+03,3276.660278,1198.972500,5.750000e+02,1.000000e+00,1.000000e+00,6.000000e+00


In [178]:
data_ml.to_csv('df.csv', index=False)

In [202]:
df = pd.read_csv('df.csv')
df.head()

,visitorid,num_items_viewed,view_count,firstview_to_cart,cart_to_transaction,bought_count,purchased,view_rate,purchase_rate
0,512286,1,1,NaN,NaN,0,0,1.000000,0.0
1,1063766,8,13,NaN,NaN,0,0,0.615385,0.0
2,58180,4,6,NaN,NaN,0,0,0.666667,0.0
3,1000730,1,1,NaN,NaN,0,0,1.000000,0.0
4,823230,1,1,NaN,NaN,0,0,1.000000,0.0


In [203]:
df['visitorid'].sort_values(ascending=True).reset_index(drop=True)

0                0
1                1
2                2
3                3
4                5
            ...   
1232595    1407575
1232596    1407576
1232597    1407577
1232598    1407578
1232599    1407579
Name: visitorid, Length: 1232600, dtype: int64

In [204]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232600 entries, 0 to 1232599
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   visitorid            1232600 non-null  int64  
 1   num_items_viewed     1232600 non-null  int64  
 2   view_count           1232600 non-null  int64  
 3   firstview_to_cart    9791 non-null     float64
 4   cart_to_transaction  9791 non-null     float64
 5   bought_count         1232600 non-null  int64  
 6   purchased            1232600 non-null  int64  
 7   view_rate            1232600 non-null  float64
 8   purchase_rate        1232600 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 84.6 MB


In [205]:
df = df.drop(['visitorid', 'view_count', 'purchased'], axis=1)

In [206]:
df.duplicated().value_counts()

True     1220816
False      11784
Name: count, dtype: int64

In [207]:
df = df.drop_duplicates()

In [208]:
len(df)

11784

In [209]:
df.head()

,num_items_viewed,firstview_to_cart,cart_to_transaction,bought_count,view_rate,purchase_rate
0,1,NaN,NaN,0,1.000000,0.0
1,8,NaN,NaN,0,0.615385,0.0
2,4,NaN,NaN,0,0.666667,0.0
5,1,NaN,NaN,0,0.500000,0.0
11,3,NaN,NaN,0,1.000000,0.0


In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11784 entries, 0 to 1232360
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   num_items_viewed     11784 non-null  int64  
 1   firstview_to_cart    9786 non-null   float64
 2   cart_to_transaction  9786 non-null   float64
 3   bought_count         11784 non-null  int64  
 4   view_rate            11784 non-null  float64
 5   purchase_rate        11784 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 644.4 KB


In [211]:
# define features
X = df.drop('purchase_rate', axis=1)
X.head()

,num_items_viewed,firstview_to_cart,cart_to_transaction,bought_count,view_rate
0,1,NaN,NaN,0,1.000000
1,8,NaN,NaN,0,0.615385
2,4,NaN,NaN,0,0.666667
5,1,NaN,NaN,0,0.500000
11,3,NaN,NaN,0,1.000000


In [212]:
# define label
y=df['purchase_rate']

In [213]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [197]:
fvc_pipeline = Pipeline([
    ('Impute_fvc', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1000))
])

cta_pipeline = Pipeline([
    ('Impute_cta', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2000))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('fvc_pipe', fvc_pipeline, ['firstview_to_cart']),
        ('cta_pipe', cta_pipeline, ['cart_to_transaction']),
        
    ], remainder='passthrough'
)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('scaler', StandardScaler()),
                       ('lr', LinearRegression())])

r2 = cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2')
mean_r2 = np.mean(r2)

print(mean_r2)

0.3983613278254092


In [125]:
# Create the pipelines for imputing and scaling
fvc_pipeline = Pipeline([
    ('Impute_fvc', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1000))
])

cta_pipeline = Pipeline([
    ('Impute_cta', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2000))
])

# Create the preprocessor column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('fvc_pipe', fvc_pipeline, ['firstview_to_cart']),
        ('cta_pipe', cta_pipeline, ['cart_to_transaction']),
    ], remainder='passthrough'
)

# Create the polynomial regression model
poly_degree = 2  # Set the degree of the polynomial
poly_reg = PolynomialFeatures(degree=poly_degree)
linear_reg = LinearRegression()

# Create the final pipeline with the polynomial regression model
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('scaler', StandardScaler()),
                       ('poly_reg', poly_reg),
                       ('linear_reg', linear_reg)])

# Fit and evaluate the pipeline using cross-validation
r2 = cross_val_score(pipe, X, y, cv=5, scoring='r2')
mean_r2 = np.mean(r2)

print(mean_r2)

0.4469561440235849


### 3.3.2 RandomForestRegressor

In [205]:
# Create the pipelines for imputing and scaling
fvc_pipeline = Pipeline([
    ('Impute_fvc', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1000))
])

cta_pipeline = Pipeline([
    ('Impute_cta', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2000))
])

# Create the preprocessor column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('fvc_pipe', fvc_pipeline, ['firstview_to_cart']),
        ('cta_pipe', cta_pipeline, ['cart_to_transaction']),
    ], remainder='passthrough'
)

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('scaler', StandardScaler()),
                       ('rf', RandomForestRegressor())])

r2 = cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2')
mean_r2 = np.mean(r2)

print(mean_r2)

0.9902997219111747


In [204]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'rf__n_estimators': [50, 100, 200],  # Number of trees in the forest
    'rf__max_depth': [None, 10, 20, 30],  # Depth of trees
    'rf__min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'rf__min_samples_leaf': [1, 2, 4]  # Minimum samples required at a leaf node
}

# Set up GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=2)

# Fit the model with hyperparameter tuning
grid_search.fit(X_test, y_test)

# Print R² scores for all iterations
print("\nAll R² Scores for Each Hyperparameter Combination:")
for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
    print(f"R² Score: {mean_score:.5f} | Params: {params}")

# Print the best parameters and best score
print("\nBest Hyperparameters:", grid_search.best_params_)
print("Best R² Score:", grid_search.best_score_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits

All R² Scores for Each Hyperparameter Combination:
R² Score: 0.95919 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 50}
R² Score: 0.96149 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
R² Score: 0.96124 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
R² Score: 0.96073 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 50}
R² Score: 0.96205 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
R² Score: 0.96167 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 200}
R² Score: 0.95723 | Params: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples

In [192]:
X_train.shape

(7895, 5)

In [ ]:
fvc_pipeline = Pipeline([
    ('Impute_fvc', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1000))
])

cta_pipeline = Pipeline([
    ('Impute_cta', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2000))
])

# Step 4: Column transformer for feature preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('fvc_pipe', fvc_pipeline, ['firstview_to_cart']),
        ('cta_pipe', cta_pipeline, ['cart_to_transaction']),
    ], remainder='passthrough'
)

# Step 5: Define the full pipeline with RandomForestRegressor
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_leaf=1,
        min_samples_split=2,
        random_state=42
    ))
])

# Step 6: Fit the model on the training data
pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Predictions
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

# Performance Metrics
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Print results
print(f"Train R²: {r2_train:.4f}")
print(f"Test R²: {r2_test:.4f}")

print(f"Train MAE: {mae_train:.6f}")
print(f"Test MAE: {mae_test:.6f}")

print(f"Train RMSE: {rmse_train:.6f}")
print(f"Test RMSE: {rmse_test:.6f}")


In [322]:
import joblib

# Save the trained model
joblib.dump(pipe, "purchasing_probability_model.pkl")

print("Model saved successfully!")


Model saved successfully!


In [121]:
import joblib

# Load the trained model
pipe = joblib.load("purchasing_probability_model.pkl")

print("Model loaded successfully!")

Model loaded successfully!


In [328]:
from sklearn.model_selection import cross_val_predict

# Create the pipelines for imputing
fvc_pipeline = Pipeline([
    ('Impute_fvc', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1000))
])

cta_pipeline = Pipeline([
    ('Impute_cta', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=2000))
])

# Preprocessor for handling missing values
preprocessor = ColumnTransformer(
    transformers=[
        ('fvc_pipe', fvc_pipeline, ['firstview_to_cart']),
        ('cta_pipe', cta_pipeline, ['cart_to_transaction']),
    ], remainder='passthrough'
)

# Define the final pipeline with best hyperparameters
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_leaf=1,
        min_samples_split=2,
        random_state=42
    ))
])


# Perform cross-validation for R²
r2_scores = cross_val_score(pipe, X, y, cv=5, scoring='r2')

# Perform cross-validation to get predictions for calculating other metrics
y_pred = cross_val_predict(pipe, X, y, cv=5)

# Calculate evaluation metrics for cross-validation results
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y - y_pred) / y)) * 100  # In percentage

# Print the cross-validation results
print(f"Cross-Validation Results:")
print(f"R² Scores for each fold: {r2_scores}")
print(f"Mean R²: {np.mean(r2_scores)}")

print(f"\nMean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")

Cross-Validation Results:
R² Scores for each fold: [0.99562704 0.99866418 0.99799926 0.99851304 0.9954875 ]
Mean R²: 0.9972582041973315

Mean Absolute Error (MAE): 3.598299989402523e-05
Mean Squared Error (MSE): 8.985496999038403e-06
Root Mean Squared Error (RMSE): 0.0029975818586050994
Mean Absolute Percentage Error (MAPE): 2.0741115325017447%
